# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd
import os
cwd = os.getcwd()

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# enter your code here
orders = pd.read_csv(cwd + "\\Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
len(orders['CustomerID'].unique())

4339

In [4]:
# Aggregate amount-spent by unique customer using grouby
aggreg_spent = orders.groupby('CustomerID').sum('amount_spent')['amount_spent']

In [5]:
# Identify quantile range by using rank(pct=True)
range_spent = orders.groupby('CustomerID').sum('amount_spent')['amount_spent'].rank(pct=True)

In [6]:
# Identify VIP with + 99 quantile and Preferred +90
range_spent_label = range_spent.apply(lambda x: 'VIP' if x >= 0.99 else 'Preferred' if x >= 0.95 else 'Standard')

In [7]:
range_spent_label.value_counts()

Standard     4122
Preferred     173
VIP            44
Name: amount_spent, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

In [8]:
# unique customers and country
cust_count = orders.groupby(['CustomerID', 'Country']).count().reset_index()

In [9]:
# merge the two dataframes
merge = pd.merge(cust_count, range_spent_label, on='CustomerID')

In [10]:
# pivot table with countryies as index and customer tag as column
merge2 = merge.pivot_table(index='Country', columns='amount_spent_y', values='CustomerID', aggfunc='count', fill_value=0)

In [11]:
# check number of customers
merge2.sum()

amount_spent_y
Preferred     174
Standard     4129
VIP            44
dtype: int64

In [12]:
# numer of customers is bigger than unique customers (8 more), let's check why
# use the list of customers ID from the groupby customer-country
test = orders.groupby(['CustomerID', 'Country']).count().reset_index()['CustomerID']

In [13]:
# check if there is any customer ID present in more than 1 country
repeated_customers = [i for i in test if list(test).count(i) > 1]

In [14]:
# We confirm There are 8 custoemrs present in 2 countries
cust_count[cust_count['CustomerID'].isin(repeated_customers)][['CustomerID', 'Country']]

,CustomerID,Country
20,12370,Austria
21,12370,Cyprus
39,12394,Belgium
40,12394,Denmark
58,12417,Belgium
59,12417,Spain
63,12422,Australia
64,12422,Switzerland
71,12429,Austria
72,12429,Denmark


In [15]:
# check which is the country with most VIP customer by indexing the column and using .max()
merge2[merge2['VIP'] == merge2['VIP'].max()]

amount_spent_y,Preferred,Standard,VIP
Country,,,
United Kingdom,140,3744,37


In [16]:
# Create a DataSeries with the sum of VIP and Preferred customers
sum = merge2['VIP'] + merge2['Preferred']

In [17]:
# check which is the country with most VIP and Preferred customer by filtering the max()
sum[sum == sum.max()]

Country
United Kingdom    177
dtype: int64